<a href="https://colab.research.google.com/github/123jyoti614/Trading_Strategy_Prediction/blob/main/data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bitmex
import numpy as np
import pandas as pd
from bitmex import bitmex
import math
import os.path
import time
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm_notebook

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for bravado: filename=bravado-12.0.1-py2.py3-none-any.whl size=37492 sha256=d3a7bc3f84b5f67fe30688e72f09196d0b54ecdefebb5ef0b7ca077c76283428
  Stored in directory: /root/.cache/pip/wheels/02/14/f5/8086cc8a76021701714fe4a4e55668f42ffe811465266e820d
  Created wheel for bravado-core: filename=bravado_core-6.1.1-py2.py3-none-any.whl size=67675 sha256=f74e4a75eee2571317b9cd34f72d5e7faf6d16b012b0e3cf2fac182370f4ac5e
  Stored in directory: /root/.cache/pip/wheels/b8/56/c6/3695a1daf18ee45607f1d352fca288d173bc3b1722228c6fb1
Successfully built bravado bravado-core


In [ ]:
key = 'w0Vc7Xs9AKChVqN_jHXWosp3'
secret = 'EaO2tP5muocUl6-xFMKXKlwDFRrx1DYNtOxrsyjkv1KnNFl_'

In [ ]:
symbol = "XBTUSD"
kline_size = "1h"

In [ ]:
bin_length = {"1m": 1, "15m": 15, "30m": 30, "1h": 60}
batch_size = 750
client = bitmex(test=False, api_key=key, api_secret=secret)

/usr/local/lib/python3.12/dist-packages/swagger_spec_validator/validator20.py:49: SwaggerValidationWarning: Found "$ref: #/definitions/UserPreferences" with siblings that will be overwritten. See https://stackoverflow.com/a/48114924 for more information. (path #/definitions/User/properties/preferences)
  warnings.warn(


In [ ]:
def update(symbol, kline_size, data):
    if len(data)!=0:  old = parser.parse(data["timestamp"].iloc[-1])
    old = client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    new = client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new

In [ ]:
def fetch(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data1 = pd.read_csv(filename)
    else: data1 = pd.DataFrame()
    oldest_point, newest_point = update(symbol, kline_size, data1)
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/bin_length[kline_size])
    rounds = math.ceil(available_data / batch_size)
    if rounds>0:
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data in %d rounds.' % (delta_min, symbol, available_data, kline_size, rounds))
        for round_num in tqdm_notebook(range(rounds)):
            time.sleep(1)
            new_time = (oldest_point + timedelta(minutes = round_num * batch_size * bin_length[kline_size]))
            data = client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=batch_size, startTime = new_time).result()[0]
            temp_df = pd.DataFrame(data)
            data1 = pd.concat([data1, temp_df])
    data1.set_index('timestamp', inplace=True)
    if save and rounds > 0: data1.to_csv(filename)
    print('Done')
    return data1

In [ ]:
data_final = fetch(symbol, kline_size, save=True)

/tmp/ipython-input-1582709997.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for round_num in tqdm_notebook(range(rounds)):


  0%|          | 0/121 [00:00<?, ?it/s]

Done
